Large Scale Spatial SEIRS Model Simulation with Automated Network Information Processing 
by Xiyuan Li

Western University
London, Ontario 
July 23nd, 2020

Virsion beta 1.0.2 Under Testing

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import csv

In [ ]:
n=[]
c=[]
p=[]
m = []
itemp=[]

In [ ]:
def load_matrix(file):
    count = 0
    with open(file, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',', quotechar=' ')
        for row in csvreader:
            if count == 0:
                header=row
            else:
                n.append(row[0])
                c.append(row[1])
                p.append(row[2])
                m.append(row[3:])
                itemp.append(int(row[2])-3)
                for number in range(3):
                    itemp.append(1)
                itemp.append(0)
                
            count +=1
    init = np.asarray(itemp)
    for number in range(count-1):
        itemp.append(int(p[number])+3)
    name = n
    population =np.asarray(p)
    coordinates = np.asarray(c)
    matrix = np.asarray(m)
    initial = np.asarray(itemp)
    return matrix, init, initial, name
    

In [ ]:
class objectClass(object):
    def __init__(self, number):
        self.number = number


tau, init, initial, name = load_matrix('large_scale_input_testing20.csv')
row = len(tau)
tau=np.rot90(np.fliplr(tau))
x=[]
y=[]
num = 11*row

for i in range(11*row): #elements and first derivatives
    x.append(objectClass(i).number)
for i in range(row*row): #sudo_matrix
    y.append(objectClass(i).number)

In [ ]:
j = 0
sudo_matrix=[]
while j < row:
    i = 0 #0
    while i < row:
        sudo_matrix.append(tau[j,i])
        i+=1 #1
    j+=1 #1
y = sudo_matrix

In [ ]:
#Initial Value Assignement
beta = 1.12
roe = 0.5
kappa = 0.27
gamma = 0.29
alpha=0.1
delta = 0.04
days = 200
params = beta, kappa, roe, gamma, delta
ODEmodelInitial = init

In [ ]:
def write_solution(file, count, S, E, Ic, Isc, R):
    number = 0
    with open(file,'w', newline='') as csvfile:
        
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        for i in range(row):
            #while number < days:
            csvwriter.writerow([name[i], *S[i]])
            number+=1
    
    return 

In [ ]:
def sum_organizer(sudo_matrix):
    g = []
    for i in range(row*3):
        g.append(objectClass(i).number)
    m_count = 0
    
    while m_count < row:
        count = int(row) #3
        while count >0:
            count-=1 #2
            g[m_count*3+0] = g[m_count*3+0] + x[5*count+2]*float(y[count+m_count*row])
            g[m_count*3+1] = g[m_count*3+1] + x[5*count+3]*float(y[count+m_count*row])
            g[m_count*3+2] = g[m_count*3+2] + x[5*row+count]*float(y[count+m_count*row])
        m_count+=1
        
    return g

In [ ]:
def get_tau_self(count):
    f=[]
    
    for i in range(row):
        f.append(y[int(count)+i*row])
    
    return f

In [ ]:
def ode_model(z, t, beta, kappa, roe, gamma, delta):
    
    number = 0
    while number < 5*row:
        x[number] = float(z[number])
        number+=1
    count = 0
    iteration = 0
    while count < num:
        f = get_tau_self(iteration)
        g = sum_organizer(y)
        #reference: N = S + E + Ic + Isc + R
        x[iteration+row*5] = x[iteration*5] + x[iteration*5+1] + x[iteration*5+2] + x[iteration*5+3] + x[iteration*5+4]
        n2 = 0
        sum = 0
        while n2 < row:
            sum = sum + float(f[n2])*(float(g[n2*3])+alpha*float(g[n2*3+1]))/float(g[n2*3+2])
            n2+=1
        x[row*6-1+iteration*5+1] = -beta*x[iteration*5]*((sum)) + delta*x[iteration*5+4]
        x[row*6-1+iteration*5+2] = beta*x[iteration*5]*((sum))- kappa*x[iteration*5+1]
        x[row*6-1+iteration*5+3] = roe*kappa*x[iteration*5+1] - gamma*x[iteration*5+2]
        x[row*6-1+iteration*5+4] = (1-roe)*kappa*x[iteration*5+1] - gamma*x[iteration*5+3]
        x[row*6-1+iteration*5+5] = gamma*(x[iteration*5+2] + x[iteration*5+3]) - delta*x[iteration*5+4]
        count+=11
        iteration+=1
    number2 = 0
    output = ()
    while number2 < row:
        i = 0
        while i < 5:
            i+=1
            output = output + (x[row*6-1+5*number2+i],)
        number2+=1
   
    return output

In [ ]:
def ode_solver(t, ODEmodelInitial, params):
    res = odeint(ode_model, ODEmodelInitial, t, args=(params))
    return res

In [ ]:
def main(ODEmodelInitial, params, days):
    
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, ODEmodelInitial, params)
    number = 0
    print(len(sol))
    print(len(sol[0]))
    print(len(sol[0]))
    print(sol[0])
    while number < 5*row:
        x[number] = sol[:,number]
        number+=1   
    
    base=0
    s=0
    e=0
    ic=0
    isc=0
    r=0
    ts = []
    te = []
    tic = []
    tisc= []
    tr = []
    while base < row: #20 rows
    
        ts.append(x[5*base+0].tolist())
        te.append(x[5*base+1].tolist())
        tic.append(x[5*base+2].tolist())
        tisc.append(x[5*base+3].tolist())
        tr.append(x[5*base+4].tolist())
        
        
        
        s=s+x[5*base+0] #final community combined
        e=e+x[5*base+1]
        ic=ic+x[5*base+2]
        isc=isc+x[5*base+3]
        r=r+x[5*base+4]
        
        base+=1
        
    write_solution('network_SEIIRS_simulation_result.csv', base, ts, te, tic, tisc, tr)

In [ ]:
main(ODEmodelInitial, params, days)